In [1]:
%pip install rouge_score

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24986 sha256=7ab2698c5a3fb6156f2ab57ee50b16d85109d9a036c5ae9eeb42660b4c9fc829
  Stored in directory: /Users/ajeyk/Library/Caches/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
Successfully built rouge_score

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from sklearn.feature_extraction.text import CountVectorizer
from scipy.stats import entropy
import re
import string
from rouge_score import rouge_scorer
from dotenv import load_dotenv


load_dotenv()

True

In [44]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))

def clean_text(text):
    text = re.sub(r'\S+@\S+', '', text)  # Remove emails
    text = re.sub(r'\b\d{3}[-.\s]?\d{3}[-.\s]?\d{4}\b', '', text)  # Remove phone numbers
    text = re.sub(r'\n+', ' ', text)  # Replace newlines with a space
    text = re.sub(r'[^a-zA-Z0-9\s.,!?]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

def preprocess_texts(texts):
    """Tokenize and clean a list of documents."""
    all_sentences = []
    all_tokenized_sentences = []
    
    for text in texts:
        sentences = sent_tokenize(text)  # Split into sentences
        for sentence in sentences:
            words = word_tokenize(sentence.lower())  # Lowercase & tokenize
            words = [w for w in words if w not in stop_words and w not in string.punctuation]  # Remove stopwords & punctuation
            if len(words) >= 5:  # Filter out short sentences
                all_tokenized_sentences.append(words)  
                all_sentences.append(sentence)  

    return all_tokenized_sentences, all_sentences


def preprocess_single_string(strin):
    """Tokenize and clean a list of documents."""
    all_sentences = []
    all_tokenized_sentences = []
    
    sentences = sent_tokenize(strin)  # Split into sentences
    for sentence in sentences:
        words = word_tokenize(sentence.lower())  # Lowercase & tokenize
        words = [w for w in words if w not in stop_words and w not in string.punctuation]  # Remove stopwords & punctuation
        if len(words) >= 5:  # Filter out short sentences
            all_tokenized_sentences.append(words)  
            all_sentences.append(sentence)  

    return all_tokenized_sentences, all_sentences

def train_lda(documents, num_topics=25):
    """Train an LDA model using Gensim."""
    dictionary = Dictionary(documents)
    bow_corpus = [dictionary.doc2bow(doc) for doc in documents]
    lda_model = LdaModel(bow_corpus, num_topics=num_topics, id2word=dictionary, passes=40, alpha="auto")
    return lda_model, dictionary, bow_corpus

def get_topic_distribution(lda_model, bow):
    """Get topic distribution for a document or summary."""
    topic_probs = lda_model.get_document_topics(bow, minimum_probability=1e-10)
    topic_dict = {k: v for k, v in topic_probs}  
    return np.array([topic_dict.get(k, 1e-10) for k in range(lda_model.num_topics)])

def kl_divergence(p, q):
    """Compute KL divergence between two probability distributions."""
    return entropy(p, q)

def kl_sum_per_topic(sentences, tokenized_sentences, lda_model, dictionary, summary_length=1):
    """Generate separate extractive summaries for each topic."""
    
    # Step 1: Get topic distribution for each sentence
    sentence_topic_dists = {}
    for i, sentence in enumerate(tokenized_sentences):
        bow = dictionary.doc2bow(sentence)
        if len(bow) > 0:  # Avoid empty docs
            sentence_topic_dists[sentences[i]] = get_topic_distribution(lda_model, bow)

    topic_summaries = {}

    # Step 2: Extract summaries per topic
    for topic_id in range(lda_model.num_topics):
        PD = np.zeros(lda_model.num_topics)  
        num_sentences = 0  

        # Compute PD: The topic distribution of all sentences that strongly belong to this topic
        for sent, dist in sentence_topic_dists.items():
            if dist[topic_id] > 0.1:  # Select sentences with meaningful weight for this topic
                PD += dist  
                num_sentences += 1  

        if num_sentences == 0:
            continue  

        PD /= num_sentences  # Normalize PD

        selected_sentences = []
        PS = np.ones_like(PD) * 1e-10  # Initialize uniform topic distribution

        while len(selected_sentences) < summary_length:
            best_sentence = None
            best_kl_score = float("inf")

            for sentence, topic_dist in sentence_topic_dists.items():
                if sentence in selected_sentences or topic_dist[topic_id] < 0.1:
                    continue  

                new_PS = (PS * len(selected_sentences) + topic_dist) / (len(selected_sentences) + 1)
                kl_score = kl_divergence(PD, new_PS)

                if kl_score < best_kl_score:
                    best_kl_score = kl_score
                    best_sentence = sentence

            if best_sentence:
                selected_sentences.append(best_sentence)
                PS = (PS * (len(selected_sentences) - 1) + sentence_topic_dists[best_sentence]) / len(selected_sentences)

        topic_summaries[topic_id] = selected_sentences

    return topic_summaries

In [45]:
def extract_sections(text):
    """Extract abstract and introduction from the text."""
    abstract_match = re.search(r"Abstract:(.*?)Introduction:", text, re.DOTALL)
    intro_match = re.search(r"Introduction:(.*)", text, re.DOTALL)
    
    abstract = abstract_match.group(1).strip() if abstract_match else ""
    introduction = intro_match.group(1).strip() if intro_match else ""
    
    return abstract, introduction


def compute_rouge_scores(reference, generated):
    """Compute ROUGE scores."""
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, generated)
    
    return {
        'rouge-1': scores['rouge1'].fmeasure,
        'rouge-2': scores['rouge2'].fmeasure,
        'rouge-l': scores['rougeL'].fmeasure
    }


def process_all_files(folder_path):
    """Process all valid text files in a folder."""
    valid_documents = []
    rouge_scores_list = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, "r", encoding="utf-8") as file:
                text = file.read()
            
            abstract, introduction = extract_sections(text)
            if not abstract or not introduction:
                continue  # Skip invalid files
            
            introduction = clean_text(introduction)
            valid_documents.append((abstract, introduction))

    # Preprocess all valid introductions
    introductions = [doc[1] for doc in valid_documents]
    tokenized_sentences, original_sentences = preprocess_texts(introductions)

    # Train LDA
    lda_model, dictionary, _ = train_lda(tokenized_sentences, num_topics=30)
    print(lda_model.print_topics(num_words=10))
    # Process each document individually
    for i, (abstract, introduction) in enumerate(valid_documents):
        tokenized_sentences, original_sentences = preprocess_single_string(introduction)
        summaries = kl_sum_per_topic(original_sentences, tokenized_sentences, lda_model, dictionary, summary_length=1)
        if len(summaries) == 0:
            summaries = {0: "No summary."}
        # if len(selected_sentences) == 0:
        #     selected_sentences = list(sentence_topic_dists.keys())[:2] if sentence_topic_dists else ["No meaningful summary."]
        generated_summary = " ".join([" ".join(sentences) for sentences in summaries.values()])
        print(f"\nFile number **{i + 1}**\nGolden Summary:\n{abstract}\n\nGenerated Summary:\n{generated_summary}")
        rouge_scores = compute_rouge_scores(abstract, generated_summary)
        rouge_scores_list.append(rouge_scores)


    # Compute average ROUGE scores
    avg_rouge = {
        'rouge-1': np.mean([r['rouge-1'] for r in rouge_scores_list]),
        'rouge-2': np.mean([r['rouge-2'] for r in rouge_scores_list]),
        'rouge-l': np.mean([r['rouge-l'] for r in rouge_scores_list])
    }

    return avg_rouge
    

In [46]:
FOLDER_PATH = os.environ.get("DUC_2001_SUMMARIES")

average_rouge = process_all_files(FOLDER_PATH)
print("Average ROUGE Scores:")
print(average_rouge)

[(1, '0.106*"cellrule" + 0.084*"tablecell" + 0.039*"cvjc" + 0.026*"chjl" + 0.024*"rowrule" + 0.022*"tablerow" + 0.021*"place" + 0.018*"board" + 0.013*"team" + 0.012*"wont"'), (17, '0.028*"see" + 0.025*"put" + 0.022*"director" + 0.019*"really" + 0.015*"legal" + 0.014*"moved" + 0.014*"traffic" + 0.014*"hispanic" + 0.013*"impact" + 0.013*"independent"'), (8, '0.034*"court" + 0.025*"might" + 0.025*"statement" + 0.021*"aircraft" + 0.016*"supreme" + 0.016*"thursday" + 0.015*"given" + 0.014*"difficult" + 0.013*"legislation" + 0.012*"certain"'), (9, '0.047*"term" + 0.036*"limits" + 0.016*"forces" + 0.013*"council" + 0.012*"equipment" + 0.012*"disaster" + 0.012*"britain" + 0.012*"media" + 0.011*"democratic" + 0.011*"relief"'), (4, '0.062*"oil" + 0.047*"exxon" + 0.032*"valdez" + 0.030*"spill" + 0.023*"coast" + 0.017*"guard" + 0.016*"cleanup" + 0.015*"tanker" + 0.014*"alaska" + 0.012*"sea"'), (14, '0.024*"went" + 0.023*"began" + 0.018*"days" + 0.013*"started" + 0.013*"line" + 0.012*"lead" + 0.012